In [10]:
##############comments#############
#22/11/2023 11:47pm - saves grid images in a folder in reverse order 
!pip install progressbar

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12067 sha256=40edbc3a35de536b1fc1d68ef0d2e13b8ead7eb48ca7080612cc84ff52ba6136
  Stored in directory: /Users/danielsloseris/Library/Caches/pip/wheels/8d/bb/b2/5353b966ac6f3c5e1000629a9a5f6aed41794487f551e32efc
Successfully built progressbar

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from matplotlib import pyplot as plt
import sys
import csv
import shutil


def pil_grid(images, max_horiz=np.iinfo(int).max):
    n_images = len(images)
    n_horiz = min(n_images, max_horiz)
    h_sizes, v_sizes = [0] * n_horiz, [0] * (n_images // n_horiz)
    for i, im in enumerate(images):
        h, v = i % n_horiz, i // n_horiz
        h_sizes[h] = max(h_sizes[h], im.size[0])
        v_sizes[v] = max(v_sizes[v], im.size[1])
    h_sizes, v_sizes = np.cumsum([0] + h_sizes), np.cumsum([0] + v_sizes)
    im_grid = Image.new('RGB', (h_sizes[-1], v_sizes[-1]), color='white')
    for i, im in enumerate(images):
        im_grid.paste(im, (h_sizes[i % n_horiz], v_sizes[i // n_horiz]))
    return im_grid 

In [13]:

# Step 1: Get all .npy file paths in a folder
folder_path_images = 'MRNet-v1.0/train/sagittal'  # Update this with the path to your folder containing .npy files
file_names = os.listdir(folder_path_images)
file_paths = [os.path.join(folder_path_images, file_name) for file_name in file_names if file_name.endswith('.npy')]


#Step 2 get current path and create new folder to save data in current path
path = os.path.join(os.getcwd(), folder_path_images + '_grid_images')


if os.path.isdir(path):
    print("Folder: {", os.path.basename(path), "} already exists! Create a new folder name!")
else:
    os.makedirs(path)
    print("Data will be saved in:")
    print(path)

Data will be saved in:
/Users/danielsloseris/Desktop/CMPT 340/Project/ImageClassification/MRNet-v1.0/train/sagittal_grid_images


In [14]:


#Step 3: iterate over all .npy volumes in folder and generate grids of the slices laid out in a square

count = 0
for file_path in file_paths:
    
    #load volume
    vol = np.load(file_path)
    nSlices = np.size(vol,0)#num of slices in image
    # print("num Slices:",nSlices)
    
    volName = os.path.basename(file_path)
    end = volName.find('.')
    volName = volName[:end]  

    # print(f"File Name: {os.path.basename(file_path)}")
         
    #add all slices into a single array "images", a PIL, Image object
    slices= []
    for i in range(nSlices):
        image = Image.fromarray(vol[i,:,:])
        # image = image.resize((128,128), 1)# can resize images to make smaller and reduce compute
        slices.append(image)
        
        
    gridEdgeSize = int(np.sqrt(nSlices)) #will round down to smallest smaller edge size for a squrae grid
    nSlicesInGrid = gridEdgeSize**2 #number of slices to include in grid to ensure square
    nSlices_Excluded = nSlices-nSlicesInGrid #number of slices excluded to ensure perfect square
    
    
    
    #divide the number of exluded slices into 2 to remove from beginning and end, 
    #to hopefully capture more information in middle slices
    nSlices_Excluded_first_half = nSlices_Excluded//2
    nSlices_Excluded_second_half = nSlices_Excluded-nSlices_Excluded_first_half
    start = nSlices_Excluded_first_half
    end = nSlices-nSlices_Excluded_second_half

    # print("File Name: {} | total slices: {}, slices included: {}".format(os.path.basename(file_path),nSlices,len(slices[start:end]) ))
    
    # print("num Slices Included",len(slices[start:end]))
    #generate grid of images (slices) for one particular .npy file, ignoring the last few
    # grid = pil_grid(slices[:nSlicesInGrid],gridEdgeSize) #removes all excluded slices from the end
    grid = pil_grid(slices[start:end],gridEdgeSize) #removes half of excluded slices from beginning and half from end
    
    #depending on the values at position intVolName on the csvs (info stored in dataArray) place image in correct folder
    grid.save(os.path.join(path, 'grid{}.jpeg'.format(volName)))
    count = count+1

print("\n===================DONE=======================")
print("Num grid Images created:",count)





===================DONE=======================
Num grid Images created: 1130
